In [4]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics 
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE, RFECV, SelectKBest, chi2
import cmath
import random

# from rdkit.Chem.Draw import IPythonConsole
# from rdkit import Chem
# from rdkit.Chem import Draw

#import pubchempy as pcp

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

from sklearn.exceptions import DataConversionWarning
import warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

%matplotlib inline

file = "QSARData.csv"

df = pd.read_csv(file)

print("original shape:", df.shape)

# ## Check for missing data

def missing_data(data):
    missing_df = data.isnull().sum(axis=0).reset_index()
    missing_df.columns = ['variable', 'missing values']
    missing_df['coverage (%)']=(data.shape[0]-missing_df['missing values'])/data.shape[0]*100
    missing_df.sort_values('coverage (%)').reset_index(drop = True)
    print(missing_df.loc[missing_df['coverage (%)'] < 90.0])
    less_cov = missing_df.loc[missing_df['coverage (%)'] < 90.0]
    return missing_df, less_cov
    

missing_df, low_cov_data = missing_data(df)
low_cols = list(low_cov_data['variable'])
df_high = df.drop(columns = low_cols, axis=1)
print("shape after dropping null columns: ", df_high.shape)

original shape: (179, 321)
          variable  missing values  coverage (%)
219  Wgamma1.unity             179           0.0
220  Wgamma2.unity             179           0.0
221  Wgamma3.unity             179           0.0
229       WG.unity             179           0.0
shape after dropping null columns:  (179, 317)


In [5]:
df_high.columns.tolist()

['Unnamed: 0',
 'depVar',
 'XLogP',
 'MW',
 'LipinskiFailures',
 'nRotB',
 'MLogP',
 'nAtomLAC',
 'nAtomP',
 'nAtomLC',
 'nB',
 'nBase',
 'nAtom',
 'nAromBond',
 'naAromAtom',
 'ALogP',
 'ALogp2',
 'AMR',
 'nAcid',
 'nSmallRings',
 'nAromRings',
 'nRingBlocks',
 'nAromBlocks',
 'nRings3',
 'nRings4',
 'nRings5',
 'nRings6',
 'nRings7',
 'nRings8',
 'nRings9',
 'tpsaEfficiency',
 'Zagreb',
 'WPATH',
 'WPOL',
 'WTPT.1',
 'WTPT.2',
 'WTPT.3',
 'WTPT.4',
 'WTPT.5',
 'VAdjMat',
 'VABC',
 'TopoPSA',
 'topoShape',
 'geomShape',
 'PetitjeanNumber',
 'MDEC.11',
 'MDEC.12',
 'MDEC.13',
 'MDEC.14',
 'MDEC.22',
 'MDEC.23',
 'MDEC.24',
 'MDEC.33',
 'MDEC.34',
 'MDEC.44',
 'MDEO.11',
 'MDEO.12',
 'MDEO.22',
 'MDEN.11',
 'MDEN.12',
 'MDEN.13',
 'MDEN.22',
 'MDEN.23',
 'MDEN.33',
 'khs.sLi',
 'khs.ssBe',
 'khs.ssssBe',
 'khs.ssBH',
 'khs.sssB',
 'khs.ssssB',
 'khs.sCH3',
 'khs.dCH2',
 'khs.ssCH2',
 'khs.tCH',
 'khs.dsCH',
 'khs.aaCH',
 'khs.sssCH',
 'khs.ddC',
 'khs.tsC',
 'khs.dssC',
 'khs.aasC',
 'k

In [6]:
## Separate the dependent and independent variables 

original = df_high.copy()
y_var = original['depVar']
original.drop(columns = ['depVar','Unnamed: 0'], inplace = True, axis=1)

In [7]:
## Visualise the artemisinin structure

a = pcp.Compound.from_cid(68827)

print("Molecular formula: ", a.molecular_formula)
print("Isomeric SMILES: ", a.isomeric_smiles)
print("Canonical SMILES: ", a.canonical_smiles)
print("XLogP: ", a.xlogp)

m = Chem.MolFromSmiles(a.isomeric_smiles)
m

NameError: name 'pcp' is not defined

In [8]:
## Removed highly correlated predictors

print("original shape: ", original.shape)

# Correlation matrix analysis
corr_matrix = original.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.8
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
print("number of features with correlations over 0.8: ", len(to_drop))

# Drop columns 
dropped = original.drop(columns = to_drop, axis=1)

print("updated shape:", dropped.shape)   

dropped.head(4)

original shape:  (179, 315)
number of features with correlations over 0.8:  173
updated shape: (179, 142)


,XLogP,MW,LipinskiFailures,nRotB,nAtomLAC,nAtomP,nBase,ALogP,nAcid,nSmallRings,...,WD.unity,BCUTw.1l,BCUTc.1l,BCUTc.1h,BCUTp.1l,PNSA.1,PNSA.3,RPCG,RPCS,RNCS
0,3.039,282.146724,0,0,0,3,0,0.8012,0,5,...,1.474806,11.999,-0.314393,0.267195,4.745096,124.877588,-26.548908,0.168942,0.185955,2.032461
1,5.687,386.209324,1,5,4,6,0,1.3625,0,6,...,1.307998,11.999,-0.314125,0.264376,4.471269,197.510368,-29.398959,0.132376,0.349696,0.963157
2,2.746,268.131074,0,0,0,3,0,0.5412,0,5,...,1.473248,11.999,-0.314390,0.264340,4.430722,135.776837,-30.360719,0.172827,0.586998,2.523145
3,3.039,282.146724,0,0,0,3,0,0.8012,0,5,...,1.474258,11.999,-0.314393,0.267195,4.745096,124.671077,-26.576362,0.168942,0.185955,2.032461


In [9]:
## Variance threshold feature selection

# Get feature variables that have zero variance for all samples

print("original shape: ", dropped.shape)
sel = VarianceThreshold(threshold=(0))
sel.fit_transform(dropped)

# Keep only columns that have a greater variance than 0.0 for all data samples 

dropped_sel = dropped[dropped.columns[sel.get_support(indices=True)]]
print("processed shape: ", dropped_sel.shape)
dropped_sel.head(4)



original shape:  (179, 142)
processed shape:  (179, 75)


,XLogP,MW,LipinskiFailures,nRotB,nAtomLAC,nAtomP,nBase,ALogP,nAcid,nSmallRings,...,WD.unity,BCUTw.1l,BCUTc.1l,BCUTc.1h,BCUTp.1l,PNSA.1,PNSA.3,RPCG,RPCS,RNCS
0,3.039,282.146724,0,0,0,3,0,0.8012,0,5,...,1.474806,11.999,-0.314393,0.267195,4.745096,124.877588,-26.548908,0.168942,0.185955,2.032461
1,5.687,386.209324,1,5,4,6,0,1.3625,0,6,...,1.307998,11.999,-0.314125,0.264376,4.471269,197.510368,-29.398959,0.132376,0.349696,0.963157
2,2.746,268.131074,0,0,0,3,0,0.5412,0,5,...,1.473248,11.999,-0.314390,0.264340,4.430722,135.776837,-30.360719,0.172827,0.586998,2.523145
3,3.039,282.146724,0,0,0,3,0,0.8012,0,5,...,1.474258,11.999,-0.314393,0.267195,4.745096,124.671077,-26.576362,0.168942,0.185955,2.032461


In [10]:
dropped_sel.columns

Index(['XLogP', 'MW', 'LipinskiFailures', 'nRotB', 'nAtomLAC', 'nAtomP',
       'nBase', 'ALogP', 'nAcid', 'nSmallRings', 'nRings5', 'nRings7',
       'tpsaEfficiency', 'WTPT.3', 'WTPT.5', 'topoShape', 'geomShape',
       'MDEC.11', 'MDEC.12', 'MDEC.13', 'MDEC.24', 'MDEC.33', 'MDEC.34',
       'MDEC.44', 'MDEO.11', 'MDEO.12', 'MDEO.22', 'MDEN.22', 'MDEN.23',
       'khs.dCH2', 'khs.ssCH2', 'khs.dsCH', 'khs.dssC', 'khs.aaaC', 'khs.sNH2',
       'khs.ssNH', 'khs.dsN', 'khs.aaN', 'khs.sssN', 'khs.sOH', 'khs.dO',
       'khs.aaO', 'khs.sF', 'khs.dsssP', 'khs.dS', 'khs.aaS', 'khs.ddssS',
       'khs.sCl', 'khs.sBr', 'FMF', 'SC.3', 'SC.4', 'SC.6', 'SCH.6', 'C1SP2',
       'C1SP3', 'C4SP3', 'ATSc4', 'ATSc5', 'Wlambda1.unity', 'Wlambda2.unity',
       'Wnu1.unity', 'Weta1.unity', 'Weta2.unity', 'Weta3.unity', 'WD.unity',
       'BCUTw.1l', 'BCUTc.1l', 'BCUTc.1h', 'BCUTp.1l', 'PNSA.1', 'PNSA.3',
       'RPCG', 'RPCS', 'RNCS'],
      dtype='object')

In [19]:
dropped_sel['y_var'] = y_var 

dropped_sel.to_csv("qsar_processed.csv", index=False)

/Users/nathalie.willems/anaconda3/envs/rdkit-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
